In [1]:
import sys,os
import serial
import serial.tools.list_ports
from PySide6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout,
    QComboBox, QPushButton, QTextEdit, QLabel, QLineEdit,QFileDialog
)
from PySide6.QtCore import QIODevice, QObject, Signal, QThread
from datetime import datetime  # 导入 datetime 模块用于获取时间
import csv
from smc import SMCController
from time import sleep

In [2]:
smc=SMCController()

In [ ]:
class SMCControllerApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.received_data_list = [] # 用于存储接收到的数据
        self.thread = None
        self.worker = None

    def initUI(self):
        # 创建串口选择下拉框
        self.port_combo = QComboBox()
        self.populate_port_list()
        self.baudrate_combo = QComboBox()
        self.baudrate_combo.addItems(["115200", "9600"])

        # 创建连接和断开按钮
        self.connect_button = QPushButton("连接")
        self.connect_button.clicked.connect(self.toggle_connection)
        self.estop_button = QPushButton("急停", enabled=False)
        self.estop_button.clicked.connect(self.estop)
        self.enable_button=QPushButton("使能")
        self.enable_button.clicked.connect(self.smcenable)
        self.gohome_button = QPushButton("回到原点")
        self.gohome_button.clicked.connect(self.gohome)
        

        # 创建文本框用于显示发送和接收到的数据
        self.receive_text = QTextEdit()
        self.receive_text.setReadOnly(True)

        # 创建发送区域
        self.dataset_input = QLineEdit()
        self.dataset_button = QPushButton("设置参数")
        self.dataset_button.clicked.connect(self.dataset)
        self.dataset_button.setEnabled(False)
        self.move_button = QPushButton("开始运动")
        self.move_button.clicked.connect(self.gomove)
        self.move_button.setEnabled(False)
        #self.move_input = QLineEdit()  # 输入目标位置的文本框

        # 创建保存按钮
        self.save_button = QPushButton("保存数据")
        self.save_button.clicked.connect(self.save_data_to_csv)
        self.save_button.setEnabled(False)

        # 布局设置
        button_layout = QHBoxLayout()
        button_layout.addWidget(QLabel("串口:"))
        button_layout.addWidget(self.port_combo)
        button_layout.addWidget(QLabel("波特率:"))
        button_layout.addWidget(self.baudrate_combo)
        button_layout.addWidget(self.connect_button)
        button_layout.addWidget(self.estop_button)
        button_layout.addWidget(self.save_button)
        button_layout.addWidget(self.enable_button)
        button_layout.addWidget(self.gohome_button)

        dataset_layout = QHBoxLayout()
        dataset_layout.addWidget(self.dataset_input)
        dataset_layout.addWidget(self.dataset_button)
        dataset_layout.addWidget(self.move_button)

        main_layout = QVBoxLayout()
        main_layout.addLayout(button_layout)
        main_layout.addWidget(self.receive_text)
        main_layout.addLayout(dataset_layout)

        self.setLayout(main_layout)
        self.setWindowTitle('电机连接调试程序')
        self.setGeometry(300, 300, 600, 400)
        self.show()

    def populate_port_list(self):
        ports = serial.tools.list_ports.comports()
        for port in ports:
            self.port_combo.addItem(port.device)

    def toggle_connection(self):
        sender = self.sender()
        if sender.text() == "连接":
            port = self.port_combo.currentText()
            baudrate = int(self.baudrate_combo.currentText())
            result=self.start_serial_thread(port, baudrate)
            if result==0:
                self.estop_button.setEnabled(True)
                self.save_button.setEnabled(True)
                self.dataset_button.setEnabled(True)
                self.gohome_button.setEnabled(True)
                self.enable_button.setEnabled(True)
                self.move_button.setEnabled(True)
                self.connect_button.setText("断开")
                print("连接成功")
            else:
                print("连接失败,失败代码为",result)
        else:
            ##断开连接的代码
            smc.stop(0,0,1)
            smc.write_sevon_pin(0,0,1) # 失能0轴
            self.enable_button.setText("使能")
            print(smc.board_close(0))
            self.estop_button.setEnabled(False)
            self.save_button.setEnabled(False)
            self.dataset_button.setEnabled(False)
            self.gohome_button.setEnabled(False)
            self.enable_button.setEnabled(False)
            self.move_button.setEnabled(False)
            self.connect_button.setText("连接")

    def start_serial_thread(self, port, baudrate):##连接板子
        try:
            result=smc.board_init(0,1, port, baudrate)  # 初始化控制器
            return result
        except Exception as e:
            print(f"初始化控制器时出错: {e}")
            return "wrong"

    
    def estop(self):
        try:
            smc.stop(0,0,1)
            result=smc.write_sevon_pin(0,0,1) # 失能0轴
            self.enable_button.setText("使能")
            print(result)
        except Exception as e:
            print(f"断开控制器连接时出错: {e}")
        
    def smcenable(self):
        if self.enable_button.text() == "使能":
            self.enable_button.setText("失能")
            try:
                result=smc.write_sevon_pin(0,0,0) # 使能0轴
                print(result)
            except Exception as e:
                print(f"使能控制器时出错: {e}")
                return
        elif self.enable_button.text() == "失能":
            self.enable_button.setText("使能")
            try:
                result=smc.write_sevon_pin(0,0,1) # 失能0轴
                print(result)
            except Exception as e:
                print(f"失能控制器时出错: {e}")
                return
    def gohome(self):
        try:
            smc.set_home_profile_unit(0,0,99,30000,1,1)
            smc.set_home_position_unit(0,0,1,0)
            smc.set_homemode(0,0,0,1,0,1)
            smc.home_move(0,0)
            smc.get_homemode(0,0)
            #print(result)
        except Exception as e:
            print(f"电机回零点时出错: {e}")
            return
    #def update_receive_text(self, data):
    #    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    #    # 将时间戳和数据组合后添加到显示框
    #    self.receive_text.append(f"[{timestamp}] {data}")

    def pttdataset(self):
        ptime=[]
        ppos=[]
        if self.parts[1]!="0" or self.parts[2]!="0":
            print("数据错误，头两个数据应为0")
            self.parts=[]
        else:
            for i in range(1, len(self.parts),2):
                ptime.append(int(self.parts[i]) ) # 将字符串转换为浮点数
                if i+1<len(self.parts):
                    ppos.append(int(self.parts[i+1]) ) # 将字符串转换为浮点数
            smc.ptt_table_unit(0,0,len(ptime),ptime,ppos)
            print("设置ptt参数成功")
        return

    def ptsdataset(self):
        ptime=[]
        ppos=[]
        pper=[]
        if self.parts[1]!="0" or self.parts[2]!="0":
            print("数据错误，头两个数据应为0")
            self.parts=[]
        else:
            for i in range(1, len(self.parts),3):
                ptime.append(int(self.parts[i]) ) # 将字符串转换为浮点数
                if i+1<len(self.parts):
                    ppos.append(int(self.parts[i+1]) ) # 将字符串转换为浮点数
                if i+2<len(self.parts):
                    pper.append(int(self.parts[i+2]) ) # 将字符串转换为浮点数
            smc.pts_table_unit(0,0,len(ptime),ptime,ppos,pper)
            print("设置pts参数成功")
        return
    def dataset(self):
        data = self.dataset_input.text()
        if data:
            self.parts=data.split(",")
            if self.parts[0]=='ptt':
                #设定ptt参数
                self.pttdataset()
            elif self.parts[0]=='vmove' and len(self.parts)==9:
                #定速运动
                #'vmove,起始速度，最大速度，加速时间，减速时间，停止速度，首次运行方向，循环次数,单趟运行时间'
                #推荐参数：vmove,1,10000,0.1,0.1,2,1,10,10
                smc.set_profile_unit(0,0,float(self.parts[1]),float(self.parts[2]),float(self.parts[3]),float(self.parts[4]),float(self.parts[5]))
                print("设置vmove参数成功")
                #smc.set_profile_unit(0,0,1,v,t,t,t)
            elif self.parts[0]=='pts':
                #pts
                self.ptsdataset(self.parts)

            else:
                print("请输入正确的运动模式")
                self.parts=[]
            #self.send_input.clear()
        else:
            print("不是哥们，啥也没输你逗我呢？")
            #print("请输入要发送的数据")

    
    def gomove(self):
        if self.parts[0]=='vmove':
            #定速运动
            smc.vmove(0,0,1)
            sleep(1)
            smc.stop(0,0,1)#运行离开原点
            for i in range(int(self.parts[7])):
                smc.vmove(0,0,int(self.parts[6]))
                sleep(int(self.parts[8]))
                if self.parts[6]=="1":
                    self.parts[6]="0"
                else :
                    self.parts[6]="1"
                smc.stop(0,0,1)
            print('往返定速运行结束')
        elif self.parts[0]=='ptt' or self.parts[0]=='pts':
            #pvt
            smc.pvt_move(0,1,[0])
            print('pvt运行结束')
        else:
            print("不能运动，请输入正确的运动模式")
        self.parts=[]

    def update_receive_text(self, data):
        timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        self.receive_text.append(f"[{timestamp}] {data}")
        self.received_data_list.append([timestamp, data])  # 将时间戳和数据添加到列表

    def save_data_to_csv(self):
        if not self.received_data_list:
            return 
        try:
            # 获取当前程序所在的文件夹路径
            program_folder = os.getcwd()
            # 获取当前时间并格式化为文件名
            current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
            file_name = f"{current_time}.csv"
            # 构建完整的文件路径
            file_path = os.path.join(program_folder, file_name)
            with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['时间戳', '数据'])  # 写入表头
                writer.writerows(self.received_data_list)  # 写入数据
        except Exception as e:
            print(f"保存数据时出错: {e}")

In [4]:

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = SMCControllerApp()
    sys.exit(app.exec())

连接成功
0
0
连接成功
请输入要发送的数据
0


SystemExit: 0

f:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
smc.stop(0,0,1)

9

In [ ]:
smc.write_sevon_pin(0,0,0)